In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-06-29


Found rundate 2024-06-28 10:00:03 < 2024-06-29 at commit 312ce5a3; returning commit 1cdecf2


'2024-06-29 10:00:03'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-06-29 10:00:03, NJSP was reporting 309 YTD deaths
Current YTD Deaths (2025-06-29 10:00:01-04:00): 261
Previous year YTD Deaths (adjusted; 2024-06-29 10:00:03): 308.12874477797516
Projected 2025 total: 611.3929334552447
-15.3% change


120d7a235ac37f40febf81bef1e364f6f868fe5c: FAUQStats cache miss: 2024, Sat Jun 29 10:00:03 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-05 14:12:00-05:00
12927,20,Union,2009,Linden City,NaN,State Highway 27,1.0,0.0,0.0,1.0,0.0,27,NaN,2024-01-06 08:37:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13271,06,Cumberland,0602,Commercial Twsp,NaN,County 633,1.0,1.0,0.0,0.0,0.0,633,NaN,2024-06-20 08:57:00-04:00
13274,01,Atlantic,0111,Galloway Twsp,NaN,County 561 (Alternate),1.0,1.0,0.0,0.0,0.0,561,NaN,2024-06-23 00:17:00-04:00
13276,14,Morris,1427,Mount Olive Twsp,NaN,State Highway 46 MP 24.4,1.0,1.0,0.0,0.0,0.0,46,NaN,2024-06-24 12:51:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,9,1,5,0,14
Bergen,8,1,2,0,11
Burlington,14,4,3,0,20
Camden,11,2,8,0,20
Cape May,5,1,0,0,6
Cumberland,5,2,3,0,9
Essex,6,3,10,1,20
Gloucester,8,4,2,0,11
Hudson,2,1,3,1,6


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

49.1% through the year, 50.9% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,14,31,9
Bergen,driver,9,14,8
Burlington,driver,17,31,14
Camden,driver,11,19,11
Cape May,driver,1,8,5
...,...,...,...,...
Salem,crashes,7,14,8
Somerset,crashes,6,18,7
Sussex,crashes,4,12,1


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,34,1,23,2,12
Bergen,27,1,13,2,10
Burlington,41,0,27,5,12
Camden,32,0,19,3,12
Cape May,34,0,26,2,1
Cumberland,22,0,13,6,8
Essex,47,2,16,10,25
Gloucester,33,0,20,13,6
Hudson,23,2,10,1,15


In [9]:
projected.to_csv(PROJECTED_CSV)